In [ ]:
# ! pip install -e /home/jovyan/downscaling/xclim/

In [1]:
%matplotlib inline
import xarray as xr
import scipy as sp
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import gcsfs

import dask.distributed as dd
import dask
import rhg_compute_tools.kubernetes as rhgk

/srv/conda/envs/notebook/lib/python3.8/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [2]:
from xclim import sdba

In [3]:
from xclim.sdba.adjustment import AnalogQuantilePreservingDownscaling, QuantileDeltaMapping
from xclim.sdba.utils import equally_spaced_nodes
from xclim.core.calendar import convert_calendar
from xclim import set_options

In [4]:
def convert_to_noleap_calendar(ds, target='noleap'):
    ds_noleap = convert_calendar(ds, target=target)
    return ds_noleap

In [5]:
'''tas_eager = xr.open_dataset('/home/jovyan/analog_method_prototype_wastate_era5_data.nc')
tas_eager = convert_calendar(tas_eager, target='noleap')
ref = tas_eager['tas']
hist = tas_eager['tas'] + 3
sim = tas_eager['tas'] + 5'''

"tas_eager = xr.open_dataset('/home/jovyan/analog_method_prototype_wastate_era5_data.nc')\ntas_eager = convert_calendar(tas_eager, target='noleap')\nref = tas_eager['tas']\nhist = tas_eager['tas'] + 3\nsim = tas_eager['tas'] + 5"

In [6]:
np.random.seed(0)

lon = [-99.83, -99.32, -99.79, -99.23]
lat = [42.25, 42.21, 42.63, 42.59]
# time = pd.date_range(start='1994-12-17', end='2015-01-15')
# time = pd.date_range(start='1995-01-01', end='2014-12-31')
time = xr.cftime_range(start="1994-12-17", end="2015-01-15", calendar="noleap")
temperature_ref = 15 + 8 * np.random.randn(len(time), 4, 4)
temperature_train = 15 + 8 * np.random.randn(len(time), 4, 4)
variable = "scen"

ref_fine = xr.Dataset(
     data_vars=dict(
         scen=(["time", "lat", "lon"], temperature_ref),
     ),
     coords=dict(
         time=time,
         lon=(["lon"], lon),
         lat=(["lat"], lat),
         # reference_time=reference_time,
     ),
     attrs=dict(description="Weather related data."),
     )
ref_fine["scen"].attrs["units"] = 'K'

ds_train = xr.Dataset(
     data_vars=dict(
         scen=(["time", "lat", "lon"], temperature_train),
     ),
     coords=dict(
         time=time,
         lon=(["lon"], lon),
         lat=(["lat"], lat),
         # reference_time=reference_time,
     ),
     attrs=dict(description="Weather related data."),
     )

In [7]:
# ref_fine['scen']

In [8]:
# ds = xr.tutorial.load_dataset("air_temperature").resample(time="D").mean()
# temp_slice = convert_to_noleap_calendar(ds['air'][:, :2, :2], target='noleap')
'''ref_fine = convert_to_noleap_calendar(ref_fine['temperature'], target='noleap')
ds_train = convert_to_noleap_calendar(ds_train['temperature'], target='noleap')'''

"ref_fine = convert_to_noleap_calendar(ref_fine['temperature'], target='noleap')\nds_train = convert_to_noleap_calendar(ds_train['temperature'], target='noleap')"

In [9]:
# ds_train

In [10]:
# temp_slice_mean = temp_slice.mean(['lat', 'lon'])
ds_ref_coarse = ref_fine.mean(['lat', 'lon'])
ds_train = ds_train.mean(['lat', 'lon'])
# temp_slice_mean_resampled = temp_slice_mean.broadcast_like(temp_slice)
ref_coarse = ds_ref_coarse.broadcast_like(ref_fine)
# temp_slice_mean_climatechange = temp_slice_mean_resampled + 3
ds_bc = ds_train + 3

ref_coarse["scen"].attrs["units"] = 'K'
ds_ref_coarse["scen"].attrs["units"] = 'K'
ds_train["scen"].attrs["units"] = 'K'
ds_bc["scen"].attrs["units"] = 'K'

In [11]:
with set_options(sdba_extra_output=True):
    quantiles = equally_spaced_nodes(620, eps=None)
    QDM = QuantileDeltaMapping.train(ds_ref_coarse["scen"], ds_train["scen"], kind='+', nquantiles=quantiles, 
                               group=sdba.Grouper("time.dayofyear", window=31))
    # QDM.train(temp_slice_mean + 2, temp_slice_mean)
    # QDM.train(ds_ref_coarse, ds_train)
    # fake_biascorrected = QDM.adjust(temp_slice_mean + 4)
    fake_biascorrected = QDM.adjust(ds_bc["scen"])
    fake_biascorrected = fake_biascorrected['scen'].assign_coords(sim_q=fake_biascorrected.sim_q).to_dataset()

In [12]:
'''print(ds_ref_coarse)
print(ds_train)
print(ds_bc)'''
# print(fake_biascorrected)

'print(ds_ref_coarse)\nprint(ds_train)\nprint(ds_bc)'

In [13]:
# biascorrected = fake_biascorrected['scen'].broadcast_like(temp_slice)
biascorrected = fake_biascorrected['scen'].broadcast_like(ref_fine)
biascorrected["sim_q"] = fake_biascorrected['sim_q'].broadcast_like(ref_fine)
year = 2005
biascorrected_year = biascorrected.sel(time=slice("{}-01-01".format(year), "{}-12-31".format(year)))
biascorrected_year = biascorrected_year.assign_coords(sim_q=biascorrected_year.sim_q).to_dataset()

In [14]:
# biascorrected_year["scen"]

In [15]:
AIQPD = AnalogQuantilePreservingDownscaling.train(ref=ref_coarse['scen'], 
                                                  hist=ref_fine['scen'], kind='+', nquantiles=int(620), 
                                                  group=sdba.Grouper("time.dayofyear", window=31))

'''AIQPD = AnalogQuantilePreservingDownscaling.train(ref=ref_coarse['scen'], 
                                                  hist=ref_fine['scen'], {'kind': '+', 'nquantiles': int(620), 
                                                  'group': sdba.Grouper("time.dayofyear", window=31)})'''

'AIQPD = AnalogQuantilePreservingDownscaling.train(ref=ref_coarse[\'scen\'], \n                                                  hist=ref_fine[\'scen\'], {\'kind\': \'+\', \'nquantiles\': int(620), \n                                                  \'group\': sdba.Grouper("time.dayofyear", window=31)})'

In [16]:
# biascorrected_year["scen"].drop('sim_q')

In [17]:
'''ds_test = xr.Dataset({"sim": biascorrected_year["scen"], 
                      "af": AIQPD.ds.af, "sim_q": biascorrected_year.scen.sim_q, "ref_coarse_q": AIQPD.ds.ref_coarse_q})'''

ds_test = xr.Dataset({"sim": biascorrected_year["scen"].drop('sim_q'), 
                      "af": AIQPD.ds.af, "sim_q": biascorrected_year.scen.sim_q, "ref_coarse_q": AIQPD.ds.ref_coarse_q})

In [18]:
# ds_test

In [19]:
# aiqpd_downscaled = AIQPD.adjust(biascorrected)
aiqpd_downscaled = AIQPD.adjust(biascorrected_year["scen"])

In [20]:
aiqpd_downscaled

<xarray.DataArray 'scen' (lon: 4, time: 365, lat: 4)>
array([[[15.56018454, 25.0351381 , 19.46581122, 20.40543795],
        [19.88532988, 21.50093264, 32.48568965, 25.73045065],
        [17.10273569, 14.54645606, 12.21171904, 15.66753275],
        ...,
        [11.69954029, 16.43709874, 17.62883181, 20.93797062],
        [11.13565763, 13.88428021, 26.68001593, 21.42118727],
        [14.82490971, 14.6196171 , 24.15682879, 24.10213445]],

       [[28.33823362, 17.17996039, 14.14056723, 23.19140451],
        [12.18405001, 31.63784109, 17.53394933, 11.00612086],
        [15.38177335, 18.47042092, 18.83244855, 10.95564698],
        ...,
        [13.33943485,  8.19617131, 13.3190681 , 22.94168295],
        [12.45080539, 29.47125594, 32.6671893 , 19.09060382],
        [16.95392835, 25.91156292, 19.30079561, 15.77585833]],

       [[11.7951727 , 11.29910936, 30.73478249, 10.33294585],
        [22.3254122 , 27.08090258, 14.11421237, 19.99852943],
        [23.90261442, 23.43500433, 28.70070918,  4.48492774],
        ...,
        [21.30143479, 18.07098234, 18.48886504, 17.22439403],
        [26.91100808, 14.56727445, 23.50561848, 29.41120573],
        [34.06932714, 22.92547502, 18.89427567, 10.15973255]],

       [[23.94326634,  9.12417649,  1.83477836, 22.19402338],
        [25.51260147, 21.93026227, 11.77232565,  8.37498556],
        [17.50571346, 15.00955063, 14.63114846, 24.40081937],
        ...,
        [14.14546734, 22.56057417, 24.07684325, 24.11779569],
        [ 9.72717095, 18.43029929, 27.32958069,  7.32852625],
        [11.60541599, 30.05809448, 28.90143484, 13.21679741]]])
Coordinates:
  * time     (time) object 2005-01-01 00:00:00 ... 2005-12-31 00:00:00
  * lat      (lat) float64 42.25 42.21 42.63 42.59
  * lon      (lon) float64 -99.83 -99.32 -99.79 -99.23
Attributes:
    units:            K
    history:          [2021-10-29 23:45:02] : Bias-adjusted with QuantileDelt...
    bias_adjustment:  AnalogQuantilePreservingDownscaling(group=Grouper(add_d...

In [ ]:
# AIQPD

In [ ]:
'''print(ds['temperature'].shape)
print(temp_slice_mean_resampled.shape)

print(20*365)
print(31*20)

print(temp_slice.shape)
print(temp_slice_mean_resampled.shape)'''

In [ ]:
# AIQPD.train(ref[:5000, :, :], hist[:5000, :, :])
# AIQPD.train(temp_slice_mean_resampled, temp_slice)
'''AIQPD.train(ref_coarse, ref_fine)
AIQPD'''

In [ ]:
print(AIQPD.ds.af.shape)

In [ ]:
AIQPD.ds.af

In [ ]:
'''len(np.unique(biascorrected.time.dt.year))
365*20'''

In [ ]:
# AIQPD.ds.af[0, 0, 0, :].isel(dayofyear=352)
# AIQPD.ds.af[0, 0, :, :].isel(dayofyear=362)

In [ ]:
biascorrected_year.chunks

In [ ]:
AIQPD.ds.af.shape

In [ ]:
AIQPD.ds.af

In [ ]:
# aiqpd_downscaled = AIQPD.adjust(biascorrected)
aiqpd_downscaled = AIQPD.adjust(biascorrected_year)

In [ ]:
sim_year_slice = ds_bc.sel(time=slice("{}-01-01".format(year), "{}-12-31".format(year)))

# pick a day
# print("bias corrected value is {}".format(temp_slice_mean_climatechange.isel(time=100).values[0][0]))
print("simulation value is {}".format(sim_year_slice.isel(time=100).values))
quantile = 0.75

print("test quantile is {}".format(quantile))

closest_quantile_index = np.argmin(np.abs(AIQPD.ds.af.quantiles.values - quantile))

'''downscaled = (AIQPD.ds.af.isel(dayofyear=100).isel(quantiles=closest_quantile_index) 
              + temp_slice_mean_climatechange.isel(time=100))'''

'''downscaled = (AIQPD.ds.af.isel(dayofyear=100).isel(quantiles=closest_quantile_index) 
              + ds_bc.isel(time=100))'''
downscaled = (AIQPD.ds.af.isel(dayofyear=100).isel(quantiles=closest_quantile_index) 
              + biascorrected_year.isel(time=100))

print("klugey downscaled mean value (where I apply adjustment factors to the bias corrected values from above) is {}".format(downscaled.mean().values))
# print("fake bias corrected value is {}".format(biascorrected.isel(time=100).values[0][0]))

print("fake bias corrected value is {}".format(biascorrected_year.isel(time=100).values[0][0]))

print("aiqpd adjusted downscaled values are {}".format(aiqpd_downscaled.isel(time=100)))

print("aiqpd adjusted downscaled value is {}".format(aiqpd_downscaled.isel(time=100).mean().values))